# MNIST Tutorial
simple reproduction

borrowed from : https://nextjournal.com/gkoehler/pytorch-mnist


In [1]:
import os
os.chdir(os.environ["WORKDIR"])

import torch
import torchvision
from matplotlib.pyplot import imshow

%matplotlib inline

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
torch.cuda.get_device_name()

'GeForce RTX 3090'

In [4]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)


In [6]:
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(
        "./datasets/",
        train=True,
        transform=torchvision.transforms.Compose(
            [
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,)),
            ]
        ),
    ),
    batch_size=batch_size_train,
    shuffle=True,
)

test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(
        "./datasets/",
        train=False,
        transform=torchvision.transforms.Compose(
            [
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,)),
            ]
        ),
    ),
    batch_size=batch_size_test,
    shuffle=True,
)


In [7]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

In [8]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [9]:
os.makedirs("./logs/MNIST/reproduction/",exist_ok=True)
def train(epoch):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )
            train_losses.append(loss.item())
            train_counter.append(
                (batch_idx * 64) + ((epoch - 1) * len(train_loader.dataset))
            )
            torch.save(network.state_dict(), "./logs/MNIST/reproduction/model.pth")
            torch.save(optimizer.state_dict(), "./logs/MNIST/reproduction/optimizer.pth")


In [10]:
def test():
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = network(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print(
        "\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )


In [11]:
test()
for epoch in range(1, n_epochs + 1):
    train(epoch)
    test()

<ipython-input-5-6c2218f37724>:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
/root/pytorch-tutorial/.venv/lib/python3.8/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3316, Accuracy: 1137/10000 (11%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.351439
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.301878
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.324649
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.280417
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.272413
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.217102
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.207311
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.160544
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.129424
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.058791
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.121551
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.902820
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.620633
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.728186
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.781542
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.545180
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.453522
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.229606
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.233822
Train Epoch: 1 [12160